In [1]:
from dotenv import load_dotenv
import os 
load_dotenv()

os.getenv('NEO4J_URI')

'neo4j+s://2f44aa74.databases.neo4j.io'

In [2]:
from langchain_community.graphs import Neo4jGraph 
# print(os.getenv('NEO4J_URI'), os.getenv('NEO4J_USERNAME'), os.getenv('NEO4J_PASSWORD'))
graph = Neo4jGraph(url=os.getenv('NEO4J_URI'), username=os.getenv('NEO4J_USERNAME'), password = os.getenv('NEO4J_PASSWORD'))
result = graph.query("MATCH (n) RETURN n LIMIT 5")
print(result)

C:\Users\indra\AppData\Local\Temp\ipykernel_28600\696776512.py:3: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph(url=os.getenv('NEO4J_URI'), username=os.getenv('NEO4J_USERNAME'), password = os.getenv('NEO4J_PASSWORD'))


[{'n': {}}, {'n': {'POB': 'SA', 'name': 'Elon Musk', 'YOB': 1967}}, {'n': {'name': 'Tesla', 'founded': 2003}}, {'n': {}}, {'n': {}}]


In [13]:
# Dataset movie 
movie_query=""" 
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' AS row 
MERGE(m: Movie {id: row.movieId})
SET m.released =date(row.released),
    m.title = row.title, 
    m.imdbRating = toFloat(row.imdbRating)
FOREACH (director IN split(row.director, '|') | 
    MERGE (p: Person {name: trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
FOREACH (actor IN split(row.actors, '|') | 
    MERGE (p: Person {name: trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m)
)
FOREACH (genre IN split(row.genres, '|') | 
    MERGE (g: Genre {name: trim(genre)})
    MERGE (g)-[:IN_GENRE]->(m)
)
"""
movie_query

" \nLOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' AS row \nMERGE(m: Movie {id: row.movieId})\nSET m.released =date(row.released),\n    m.title = row.title, \n    m.imdbRating = toFloat(row.imdbRating)\nFOREACH (director IN split(row.director, '|') | \n    MERGE (p: Person {name: trim(director)})\n    MERGE (p)-[:DIRECTED]->(m))\nFOREACH (actor IN split(row.actors, '|') | \n    MERGE (p: Person {name: trim(actor)})\n    MERGE (p)-[:ACTED_IN]->(m)\n)\nFOREACH (genre IN split(row.genres, '|') | \n    MERGE (g: Genre {name: trim(genre)})\n    MERGE (g)-[:IN_GENRE]->(m)\n)\n"

In [14]:
graph.query(movie_query)

[]

In [15]:
graph.refresh_schema()
graph.schema

'Node properties:\nCEO {POB: STRING, name: STRING, YOB: INTEGER}\nCompany {name: STRING, founded: INTEGER}\nPerson {name: STRING, YOB: INTEGER, DOB: INTEGER}\nMovie {title: STRING, released: INTEGER, id: STRING, imdbRating: FLOAT}\nUser {name: STRING, city: STRING, userId: INTEGER, age: INTEGER}\nPost {postId: INTEGER, content: STRING, timestamp: DATE_TIME}\nGenre {name: STRING}\nRelationship properties:\n\nThe relationships:\n(:Person)-[:ACTED_IN]->(:Movie)\n(:Person)-[:DIRECTED]->(:Movie)\n(:User)-[:POSTED]->(:Post)\n(:User)-[:LIKES]->(:User)\n(:User)-[:FRIEND]->(:User)\n(:Genre)-[:IN_GENRE]->(:Movie)'

In [16]:
from langchain_groq import ChatGroq 
llm = ChatGroq(groq_api_key = os.getenv("GROQ_API_KEY"), model="gemma2-9b-it")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000018B47F13370>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000018B47F48820>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [18]:
from langchain.chains import GraphCypherQAChain 
chain = GraphCypherQAChain.from_llm(graph=graph, llm=llm, verbose=True, allow_dangerous_requests=True)
chain


GraphCypherQAChain(verbose=True, graph=<langchain_community.graphs.neo4j_graph.Neo4jGraph object at 0x0000018B32E39900>, cypher_generation_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question', 'schema'], input_types={}, partial_variables={}, template='Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nThe question is:\n{question}'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000018B47F13370>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000018B47F48820>, model_name=

In [21]:
response = chain.invoke({
    # "query": "Which movies did Tom Hanks act in?"
    "query": "Who directed the Casino movies?"
})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person)-[:DIRECTED]->(m:Movie {title:"Casino"}) RETURN p.name
Full Context:
[{'p.name': 'Martin Scorsese'}]

> Finished chain.


In [31]:
examples = [
    {
        "question": "How many artists are there?",
        "query": "MATCH (a:Person)-[:ACTED_IN]->(:Movie) RETURN count(DISTINCT a)",
    },
    {
        "question": "Which actors played in the movie Casino?",
        "query": "MATCH (m:Movie {{title: 'Casino'}})<-[:ACTED_IN]-(a) RETURN a.name",
    },
    {
        "question": "How many movies has Tom Hanks acted in?",
        "query": "MATCH (a:Person {name: 'Tom Hanks'})-[:ACTED_IN]->(m:Movie) RETURN count(m)",
    },
    {
        "question": "List all the genres of the movie Schindler's List",
        "query": "MATCH (m:Movie {{title: 'Schindler\\'s List'}})-[:IN_GENRE]->(g:Genre) RETURN g.name",
    },
    {
        "question": "Which actors have worked in movies from both the comedy and action genres?",
        "query": "MATCH (a:Person)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g1:Genre), (a)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g2:Genre) WHERE g1.name = 'Comedy' AND g2.name = 'Action' RETURN DISTINCT a.name",
    },
    {
        "question": "Which directors have made movies with at least three different actors named 'John'?",
        "query": "MATCH (d:Person)-[:DIRECTED]->(m:Movie)<-[:ACTED_IN]-(a:Person) WHERE a.name STARTS WITH 'John' WITH d, COUNT(DISTINCT a) AS JohnsCount WHERE JohnsCount >= 3 RETURN d.name",
    },
    {
        "question": "Identify movies where directors also played a role in the film.",
        "query": "MATCH (p:Person)-[:DIRECTED]->(m:Movie), (p)-[:ACTED_IN]->(m) RETURN m.title, p.name",
    },
    {
        "question": "Find the actor with the highest number of movies in the database.",
        "query": "MATCH (a:Actor)-[:ACTED_IN]->(m:Movie) RETURN a.name, COUNT(m) AS movieCount ORDER BY movieCount DESC LIMIT 1",
    },
]

In [34]:
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate 

example_prompt = PromptTemplate.from_template(
    "User input: {question}\n Cypher query: {query}"
)
prompt = FewShotPromptTemplate(
    examples= examples[:5],
    example_prompt=example_prompt,
    prefix="You are a Neo4j expert. Given a schema {schema} and an input question, create a syntatically accurate Cypher query",
    suffix="User question: {question} \n Cypher query:",
    input_variables=["question", "schema"]
)
prompt

FewShotPromptTemplate(input_variables=['question', 'schema'], input_types={}, partial_variables={}, examples=[{'question': 'How many artists are there?', 'query': 'MATCH (a:Person)-[:ACTED_IN]->(:Movie) RETURN count(DISTINCT a)'}, {'question': 'Which actors played in the movie Casino?', 'query': "MATCH (m:Movie {{title: 'Casino'}})<-[:ACTED_IN]-(a) RETURN a.name"}, {'question': 'How many movies has Tom Hanks acted in?', 'query': "MATCH (a:Person {name: 'Tom Hanks'})-[:ACTED_IN]->(m:Movie) RETURN count(m)"}, {'question': "List all the genres of the movie Schindler's List", 'query': "MATCH (m:Movie {{title: 'Schindler\\'s List'}})-[:IN_GENRE]->(g:Genre) RETURN g.name"}, {'question': 'Which actors have worked in movies from both the comedy and action genres?', 'query': "MATCH (a:Person)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g1:Genre), (a)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g2:Genre) WHERE g1.name = 'Comedy' AND g2.name = 'Action' RETURN DISTINCT a.name"}], example_prompt=PromptTemplate(inp

In [ ]:
res = prompt.format(
    question="How many movies are there?",
    schema="foo"
)
res